# <h1 align="center"><font color="gree">Projeto: Regressão Linear Múltipla</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este notebook mostrará como criar e consultar uma ``tabela`` ou ``DataFrame`` que você enviou para o ``DBFS``. O [DBFS](https://docs.databricks.com/aws/pt/dbfs) é o ``Sistema de Arquivos do Databricks`` que permite armazenar dados para consulta dentro do Databricks. Este notebook assume que você já possui um arquivo dentro do DBFS que deseja ler.

Este notebook está escrito em ``Python``, então o tipo de célula padrão é Python. No entanto, você pode usar diferentes linguagens utilizando a sintaxe %LANGUAGE. ``Python``, ``Scala``, ``SQL`` e ``R`` são todos suportados.

## <font color="red">⚙️ Configurações para o Databricks</font>

In [1]:
# Só executar quando estiver trabalhando localmente:
import os
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())

databricks_host = os.environ['DATABRICKS_HOST']
databricks_token = os.environ['DATABRICKS_TOKEN']
databricks_cluster_id = os.environ['DATABRICKS_CLUSTER_ID']

print("🔗 Carregado minhas credenciais do Databricks, com sucesso!")

🔗 Carregado minhas credenciais do Databricks, com sucesso!


A seguir vamos usar `Databricks Connect` para conectar ao Databricks e executar código `Spark` localmente, mas processando os dados remotamente no cluster Databricks.

In [2]:
from databricks.connect import DatabricksSession # Esta classe é usada para conectar ao Databricks remotamente
from pyspark.dbutils import DBUtils # Para usar dbutils localmente

spark = DatabricksSession.builder.remote(host=databricks_host, token=databricks_token, cluster_id=databricks_cluster_id).getOrCreate()

# Criar instância do DBUtils para usar dbutils localmente:
dbutils = DBUtils(spark)

print("✅ Conectado ao Databricks com sucesso!")
print(f"🔧 Versão Spark: {spark.version}")
print(f"📦 DBUtils configurado e pronto para uso!")

✅ Conectado ao Databricks com sucesso!
🔧 Versão Spark: 4.0.0
📦 DBUtils configurado e pronto para uso!


In [3]:
# Executar, só lá, no Databricks para conhecer o ID do Cluster:

#databricks_cluster_id = spark.conf.get("spark.databricks.clusterUsageTags.clusterId")

#print(f"Databricks Cluster ID: {databricks_cluster_id}")

## <font color="red">📝 Resumo de nosso Dataset</font>

Nosso Dataset contém informações sobre **gorjetas de um restaurante**, com as seguintes características:

**Variáveis:**
- **`total_bill`** (numérica): Valor total da conta em dólares
- **`tip`** (numérica): Valor da gorjeta em dólares
- **`sex`** (categórica): Sexo do cliente (Male/Female)
- **`smoker`** (categórica): Se o cliente é fumante (Yes/No)
- **`day`** (categórica): Dia da semana (Sun, Sat, Thur, Fri)
- **`time`** (categórica): Período da refeição (Lunch/Dinner)
- **`size`** (numérica/ordinal): Tamanho do grupo (número de pessoas)

**Objetivo do Modelo:**
Prever o valor da **conta total** (`total_bill`) com base nas outras variáveis, usando **Regressão Linear Múltipla**.


In [10]:
file_location = "/Volumes/cat_multiple_linear_regression/schema_ml_mult_regression/vol_ml_regression_csv"

# As opções aplicadas são para arquivos CSV. Para outros tipos de arquivos, elas serão ignoradas:
df =spark.read.csv(file_location, header=True, inferSchema=True)

df.show(10)
#display(df)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 10 rows


In [11]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



## <font color="red">📊 EDA (Exploratory Data Analysis)</font>

A seguir vou mostrar um resumo estatístico do Dataset de gorjetas (``tips``) de um restaurante.

In [ ]:
# 1. Dimensões do Dataset:
print(f"Número de registros: {df.count()}")
print(f"Número de colunas: {len(df.columns)}")
print(f"Colunas: {df.columns}")


📌 DIMENSÕES DO DATASET
Número de registros: 244
Número de colunas: 7
Colunas: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']


In [ ]:
# 2. Estatísticas Descritivas das Variáveis Numéricas
print("\n" + "=" * 60)
print("📊 ESTATÍSTICAS DESCRITIVAS")
print("=" * 60)
df.describe().show()


In [ ]:
# 3. Análise das Variáveis Categóricas
print("\n" + "=" * 60)
print("🏷️  VARIÁVEIS CATEGÓRICAS")
print("=" * 60)

# Sexo
print("\n📍 Distribuição por SEXO:")
df.groupBy("sex").count().orderBy("count", ascending=False).show()

# Fumante
print("📍 Distribuição por FUMANTE:")
df.groupBy("smoker").count().orderBy("count", ascending=False).show()

# Dia da semana
print("📍 Distribuição por DIA DA SEMANA:")
df.groupBy("day").count().orderBy("count", ascending=False).show()

# Período (Lunch/Dinner)
print("📍 Distribuição por PERÍODO:")
df.groupBy("time").count().orderBy("count", ascending=False).show()

# Tamanho do grupo
print("📍 Distribuição por TAMANHO DO GRUPO:")
df.groupBy("size").count().orderBy("size").show()


In [ ]:
# 4. Verificação de Valores Nulos
from pyspark.sql.functions import col, sum as spark_sum

print("\n" + "=" * 60)
print("🔍 VERIFICAÇÃO DE VALORES NULOS")
print("=" * 60)

df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()


In [ ]:
# 5. Análise de Correlação entre Gorjeta e Conta Total
from pyspark.sql.functions import corr

print("\n" + "=" * 60)
print("📈 CORRELAÇÃO ENTRE VARIÁVEIS NUMÉRICAS")
print("=" * 60)

correlation = df.stat.corr("total_bill", "tip")
print(f"\nCorrelação entre Total da Conta e Gorjeta: {correlation:.4f}")
print("\n💡 Interpretação:")
if correlation > 0.7:
    print("   ✓ Correlação FORTE e positiva - quanto maior a conta, maior a gorjeta")
elif correlation > 0.4:
    print("   ✓ Correlação MODERADA e positiva - existe relação entre conta e gorjeta")
else:
    print("   ✓ Correlação FRACA - pouca relação linear entre as variáveis")


In [12]:
df.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
### Handling Categorical Features
from pyspark.ml.feature import StringIndexer

In [0]:
df.show()

In [0]:
indexer=StringIndexer(inputCol="sex",outputCol="sex_indexed")
df_r=indexer.fit(df).transform(df)
df_r.show()

In [0]:
indexer=StringIndexer(inputCols=["smoker","day","time"],outputCols=["smoker_indexed","day_indexed",
                                                                  "time_index"])
df_r=indexer.fit(df_r).transform(df_r)
df_r.show()

In [0]:
df_r.columns

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed',
                          'time_index'],outputCol="Independent Features")
output=featureassembler.transform(df_r)

In [0]:
output.select('Independent Features').show()

In [0]:
output.show()

In [0]:
finalized_data=output.select("Independent Features","total_bill")

In [0]:
finalized_data.show()

In [0]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [0]:
regressor.coefficients

In [0]:
regressor.intercept

In [0]:
### Predictions
pred_results=regressor.evaluate(test_data)

In [0]:
## Final comparison
pred_results.predictions.show()

In [0]:
### PErformance Metrics
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError